In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles
import plotly.express as px
import seaborn as sns
from datetime import datetime

from sklearn import datasets                      # To import the sample dataset
from sklearn.preprocessing import StandardScaler  # To transform the dataset
from sklearn.cluster import KMeans                # To instantiate, train and use model
from sklearn import metrics                       # For Model Evaluation

C:\Users\Admin\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\Admin\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.ht

In [2]:
dataset = pd.read_csv('./steam_reviews.csv', header='infer', nrows=100000)
# dataset.columns

Index(['Unnamed: 0', 'app_id', 'app_name', 'review_id', 'language', 'review',
       'timestamp_created', 'timestamp_updated', 'recommended',
       'votes_helpful', 'votes_funny', 'weighted_vote_score', 'comment_count',
       'steam_purchase', 'received_for_free', 'written_during_early_access',
       'author.steamid', 'author.num_games_owned', 'author.num_reviews',
       'author.playtime_forever', 'author.playtime_last_two_weeks',
       'author.playtime_at_review', 'author.last_played'],
      dtype='object')

In [7]:
"""What are the top 3 languages used to review applications?"""

lang = dataset[dataset.timestamp_created.dt.dayofweek == 1].language

l = dict(lang.value_counts())
top3 = list(l.keys())[:3]
print(top3)

['english', 'schinese', 'russian']


In [8]:
"""Create a function that receives as parameters both the name of a data set and a list of languages’ names 
and returns a data frame filtered only with the reviews written in the provided languages"""

def review_lang(data, language_name):
    boolean_series = data.language.isin(language_name)
    data = data[boolean_series]
    return data

review_lang(dataset,top3)

,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,2021-01-23 06:00:29,2021-01-23 06:00:29,True,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,2021-01-22 19:23:03
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,2021-01-23 05:50:30,2021-01-23 05:50:30,True,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,2021-01-23 07:18:27
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,2021-01-23 05:46:40,2021-01-23 05:46:40,True,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,2021-01-23 06:36:17
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",2021-01-23 05:32:50,2021-01-23 05:32:50,True,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,2021-01-23 06:35:44
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,2021-01-23 05:23:47,2021-01-23 05:23:47,True,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,2021-01-16 09:10:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,99994,292030,The Witcher 3: Wild Hunt,69870112,schinese,very good,2020-05-27 03:16:38,2020-05-27 03:16:38,True,0,...,True,False,False,76561199016634716,5,2,6908.0,0.0,527.0,2020-07-31 10:54:55
99995,99995,292030,The Witcher 3: Wild Hunt,69870073,english,"""Witchers were made to kill monsters. It doesn...",2020-05-27 03:15:22,2020-08-05 06:20:39,True,0,...,True,False,False,76561198157432426,58,3,7685.0,0.0,5827.0,2020-09-28 17:18:08
99996,99996,292030,The Witcher 3: Wild Hunt,69869984,english,Awesome game 100%,2020-05-27 03:12:22,2020-05-27 03:12:22,True,0,...,True,False,False,76561198316783809,28,4,6743.0,0.0,4572.0,2020-10-27 01:12:58
99997,99997,292030,The Witcher 3: Wild Hunt,69869740,schinese,优秀,2020-05-27 03:04:56,2020-05-27 03:04:56,True,0,...,True,False,False,76561199053134442,5,2,407.0,0.0,191.0,2020-06-10 12:42:29


In [19]:
"""Use the function created in the previous literal to find what percentage of these reviews
(associated with the top 3 languages) were voted as funny?"""

rv = review_lang(dataset, top3)[dataset.votes_funny == 1].count().app_name/dataset.count().app_name
print("{:.0%}".format(rv))

# l = dict(lang.value_counts())
# top3 = list(l.keys())[:3]
# print(top3)

3%


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [20]:
"""Use the function created in the literal “a” to find what percentage of these reviews
(associated with the top 3 languages) were voted as helpful?"""

rv1=(review_lang(dataset, top3)[dataset.votes_helpful == 1].count().app_name/dataset.count().app_name)
print("{:.0%}".format(rv))

3%


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
